In [1]:
commits = spark.read.parquet("gs://insinyur-data/github4")
commits.createOrReplaceTempView('commits')

In [4]:
commits.printSchema()

root
 |-- committer: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- time_sec: long (nullable = true)
 |    |-- tz_offset: long (nullable = true)
 |    |-- date: timestamp (nullable = true)
 |-- message: string (nullable = true)
 |-- repo_name: string (nullable = true)



In [6]:
spark.sql(
    'SELECT committer.name, committer.date, message, repo_name \
    FROM commits').show()

+-------+-------------------+--------------------+--------------+
|   name|               date|             message|     repo_name|
+-------+-------------------+--------------------+--------------+
|       |2006-01-10 13:39:07|MIPS: Au1550: Fix...|torvalds/linux|
|       |2006-02-28 04:55:02|[SCSI] Neaten com...|torvalds/linux|
|       |2006-02-28 04:55:05|[SCSI] PCI Error ...|torvalds/linux|
|    ard|2015-12-13 18:18:29|ARM: only conside...|torvalds/linux|
|   Tixy|2011-07-13 17:32:41|ARM: kprobes: Add...|torvalds/linux|
|   root|2011-10-28 12:58:57|direct-io: use a ...|torvalds/linux|
| Tao Ma|2010-08-12 02:40:01|ocfs2: Add readah...|torvalds/linux|
| Wei Xu|2016-04-15 15:43:04|arm64: dts: hikey...|torvalds/linux|
| Wei Xu|2014-09-28 02:27:09|clk: hix5hd2: add...|torvalds/linux|
| Wei Xu|2014-09-28 02:27:04|clk: hix5hd2: add...|torvalds/linux|
|Al Viro|2011-03-23 20:36:55|mm: arch: rename ...|torvalds/linux|
|Al Viro|2016-03-31 04:30:15|posix_acl: Inode ...|torvalds/linux|
|Al Viro|2

In [11]:
spark.sql(
    'SELECT committer.name, COUNT(1) AS count \
    FROM commits \
    GROUP BY 1 \
    ORDER BY 2 DESC \
    LIMIT 10').show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|      Linus Torvalds|65331|
|  Greg Kroah-Hartman|57661|
|     David S. Miller|54669|
|Mauro Carvalho Ch...|23407|
|         Ingo Molnar|19120|
|    John W. Linville|18882|
|          Mark Brown|17526|
|     James Bottomley| 9121|
|        Takashi Iwai| 8661|
|        Russell King| 8648|
+--------------------+-----+



In [12]:
count_jun_2012 = spark.sql(
    'SELECT COUNT(committer.date) \
    FROM commits \
    WHERE committer.date BETWEEN "2012-06-01" AND "2012-06-30"')

In [13]:
count_jun_2012.collect()

[Row(count(committer.date AS `date`)=5015)]

In [16]:
count_jun_2012.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) HashAggregate(keys=[], functions=[count(_gen_alias_91#91)])
   +- ShuffleQueryStage 0
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#174]
         +- *(1) HashAggregate(keys=[], functions=[partial_count(_gen_alias_91#91)])
            +- *(1) Project [committer#0.date AS _gen_alias_91#91]
               +- *(1) Filter ((isnotnull(committer#0) AND (committer#0.date >= 1338508800000000)) AND (committer#0.date <= 1341014400000000))
                  +- FileScan parquet [committer#0] Batched: false, DataFilters: [isnotnull(committer#0), (committer#0.date >= 1338508800000000), (committer#0.date <= 13410144000..., Format: Parquet, Location: InMemoryFileIndex[gs://insinyur-data/github4], PartitionFilters: [], PushedFilters: [IsNotNull(committer), GreaterThanOrEqual(committer.date,2012-06-01 00:00:00.0), LessThanOrEqual(..., ReadSchema: struct<committer:struct<date:timestamp>>
+- == Initial Plan ==